In [1]:
import numpy as np
import itk
import os

In [2]:
# Set chosen image
chosen_im = 'copd2'
chosen_param = 'par0000'

# Set path to data folder containing copd1, copd2, etc. folders
data_dir = "../data"

#Set paths to landmarks and images
path_landmarkse = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_eBH_xyz_r1.txt")
path_landmarksi = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_iBH_xyz_r1.txt")
path_imge = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_eBHCT.mha')
path_imgi = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.mha')

# Set paths to output of transformix
path_transformix = f"../registration-results/{chosen_param}/{chosen_im}/outputpoints.txt"

In [3]:
landmarkse = np.loadtxt(path_landmarkse)
landmarksi = np.loadtxt(path_landmarksi)

pixel_type = itk.ctype('unsigned short')
imge = itk.imread(path_imge, pixel_type)


In [4]:
# Function to calculate TRE
def TRE(landmarks1, landmarks2, spacing):
    """
    mean (and standard deviation) 3D Euclidean magnitude distance 
    between calculated and reference landmark positions for the 
    set of validation landmarks. All values are reported in units of millimeters.
    
    Parameters:    
        landmarks1 (ndarry): first set of landmarks.
        landmarks2 (ndarry): second set of landmarks.
        spaing (tuple(float)): pixel spacing for x, y, z.
    
    Returns:
        (floar, float) mean and SD 3D Euclidean magnitude distance.
    """
    landmarks1 = spacing*landmarks1
    landmarks2 = spacing*landmarks2
    diff = landmarks1 - landmarks2
    squared = diff * diff
    dist = np.sqrt(np.sum(squared,axis=1))
    mean_TRE = np.mean(dist)
    sd_TRE = np.std(dist)

    return dist, mean_TRE, sd_TRE

#Import transformix text file and convert to np array
def transformix2np(path_transformix, no_points=300):
    import re
    landmarks = open(path_transformix, "r")
    reg_expr = r'OutputPoint = \[([\d.\s\-]+)\]'
    landmarks_array = np.zeros((no_points, 3))

    for index, line in enumerate(landmarks):
        match_obj = re.search(reg_expr, line, re.M)
        coords = match_obj.group(1).split()
        coords = [round(float(c)) for c in coords]
        landmarks_array[index,:] = coords
    return landmarks_array

In [7]:
dist, mean_TRE, SD_TRE = TRE(landmarkse, landmarksi, tuple(imge.GetSpacing()))
print(mean_TRE, SD_TRE)

21.64084010925038 6.415267676867441


In [8]:
transformix_landmarks = transformix2np(path_transformix, no_points=300)
dist, mean_TRE, SD_TRE = TRE(landmarksi, transformix_landmarks, tuple(imge.GetSpacing()))
print(mean_TRE, SD_TRE)

46.735037402429555 6.267673065833723
